In [1]:
import pandas as pd
from tqdm import tqdm
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import shap

df = pd.read_csv('/home/maia-user/myl/sc/Datasets/processed_total.csv')
df.head()

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.750561,1.416667e-07,0.000005,0.0,9.302326e-07,0.0,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,0.750561,1.166667e-07,0.000005,0.0,9.302326e-07,0.0,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,0.750561,1.166667e-07,0.000005,0.0,9.302326e-07,0.0,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,0.750561,1.166667e-07,0.000005,0.0,9.302326e-07,0.0,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,0.755108,1.333333e-07,0.000005,0.0,9.302326e-07,0.0,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [2]:
print(df.shape)
print(df["Label"].value_counts())

(2827876, 79)
Label
BENIGN                        2271320
DoS Hulk                       230124
PortScan                       158804
DDoS                           128025
DoS GoldenEye                   10293
FTP-Patator                      7935
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1956
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64


In [10]:
# 假设你已经加载好 df，并完成归一化，包含 "Label" 列
rare_classes = ["Infiltration", "Web Attack � XSS", "Web Attack � Sql Injection", "Heartbleed"]
# 固定 BENIGN 样本数（防止混淆 GAN 效果）
benign_df = df[df["Label"] == "BENIGN"].sample(n=10000, random_state=42)

rare_raw = df[df["Label"].isin(rare_classes)]
df_raw = pd.concat([benign_df, rare_raw], ignore_index=True)

In [11]:
from cgan_module import cgan_augment
from Acgan import acgan_augment
from evaluate import evaluate_pipeline, evaluate_on_original_rare_samples, shap_analysis_global
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# ⚙️ 模型 & 设置
models = {
    "RandomForest": lambda: RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": lambda: XGBClassifier(use_label_encoder=False, eval_metric="mlogloss"),
    "LightGBM": lambda: LGBMClassifier()
}

# ✅ 先增强两份数据
df_cgan = cgan_augment(df, rare_classes, generate_per_class=500)
df_acgan = acgan_augment(df, rare_classes, generate_per_class=500)
# ✅ 加入 BENIGN 构建完整训练集
rare_cgan = df_cgan[df_cgan["Label"].isin(rare_classes)]
df_cgan_full = pd.concat([benign_df, rare_cgan], ignore_index=True)

rare_acgan = df_acgan[df_acgan["Label"].isin(rare_classes)]
df_acgan_full = pd.concat([benign_df, rare_acgan], ignore_index=True)

# ✅ 逐个模型进行评估
results_all = []

for model_name, model_func in models.items():
    print(f"\n🚀 正在评估模型: {model_name}")

    # ---- A. Baseline ----
    clf_base, X_base, _, _, le_base = evaluate_pipeline(
        df_raw, f"{model_name} Baseline",
        rare_classes=rare_classes,
        model_cls=model_func
    )

    # ---- B. CGAN ----
    clf_cgan, X_cgan, _, _, le_cgan = evaluate_pipeline(
        df_cgan_full, f"{model_name} CGAN增强",
        rare_classes=rare_classes,
        model_cls=model_func
    )

    # ---- C. Improved CGAN ----
    clf_acgan, X_icgan, _, _, le_acgan = evaluate_pipeline(
        df_acgan_full, f"{model_name} ACGAN增强",
        rare_classes=rare_classes,
        model_cls=model_func
    )

    # --- 原始攻击识别性能评估 ---
    for label, clf, le, tag in [
        ("Baseline", clf_base, le_base, "Base"),
        ("CGAN", clf_cgan, le_cgan, "CGAN"),
        ("ACGAN", clf_acgan, le_acgan, "ACGAN")
    ]:
        rare_detect = evaluate_on_original_rare_samples(clf, df_raw, le)
        rare_detect["Model"] = model_name
        rare_detect["Enhancement"] = tag
        results_all.append(rare_detect)

# ✅ 汇总
final_df = pd.concat(results_all, ignore_index=True)
pivot_df = final_df.pivot_table(
    index=["Model", "Class", "Original Sample Count"],
    columns="Enhancement",
    values=["Detected Correctly", "Recall"]
).reset_index()

# ✅ 展示
print("\n📊 多模型 + 多增强方法 对原始 Rare 攻击识别对比：")
print(pivot_df.to_string(index=False))



🔍 正在增强类别: Infiltration
[0/500] Loss_D: 1.3871, Loss_G: 0.6971
[100/500] Loss_D: 0.8512, Loss_G: 1.4327
[200/500] Loss_D: 0.3547, Loss_G: 2.8388
[300/500] Loss_D: 0.1227, Loss_G: 3.6115
[400/500] Loss_D: 1.2290, Loss_G: 1.2476
✅ 生成完成: Infiltration → 500 条

🔍 正在增强类别: Web Attack � XSS
[0/500] Loss_D: 1.3927, Loss_G: 0.6901
[100/500] Loss_D: 0.7944, Loss_G: 1.4336
[200/500] Loss_D: 0.2969, Loss_G: 2.7773
[300/500] Loss_D: 0.9909, Loss_G: 0.9834
[400/500] Loss_D: 0.8314, Loss_G: 1.2715
✅ 生成完成: Web Attack � XSS → 500 条

🔍 正在增强类别: Web Attack � Sql Injection
[0/500] Loss_D: 1.3910, Loss_G: 0.7312
[100/500] Loss_D: 0.9120, Loss_G: 1.3764
[200/500] Loss_D: 0.2718, Loss_G: 2.9681
[300/500] Loss_D: 0.4877, Loss_G: 1.6060
[400/500] Loss_D: 0.9630, Loss_G: 1.1729
✅ 生成完成: Web Attack � Sql Injection → 500 条

🔍 正在增强类别: Heartbleed
[0/500] Loss_D: 1.4074, Loss_G: 0.6860
[100/500] Loss_D: 0.6035, Loss_G: 1.5374
[200/500] Loss_D: 0.2346, Loss_G: 3.0570
[300/500] Loss_D: 0.1005, Loss_G: 3.7141
[400/500] Lo

/home/maia-user/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [22:58:03] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



📊 分类报告 (XGBoost Baseline):
                            precision    recall  f1-score   support

                    BENIGN     0.9970    1.0000    0.9985      2000
                Heartbleed     1.0000    0.5000    0.6667         2
              Infiltration     0.8333    0.7143    0.7692         7
Web Attack � Sql Injection     1.0000    0.7500    0.8571         4
          Web Attack � XSS     1.0000    0.9771    0.9884       131

                  accuracy                         0.9967      2144
                 macro avg     0.9661    0.7883    0.8560      2144
              weighted avg     0.9967    0.9967    0.9966      2144


📌 Rare 类别测试集表现：
                     Class  Test Samples  Correctly Predicted  Recall
              Infiltration             7                    5  0.7143
          Web Attack � XSS           131                  128  0.9771
Web Attack � Sql Injection             4                    3  0.7500
                Heartbleed             2                    

/home/maia-user/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [22:58:03] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



📊 分类报告 (XGBoost CGAN增强):
                            precision    recall  f1-score   support

                    BENIGN     0.9950    1.0000    0.9975      2000
                Heartbleed     1.0000    1.0000    1.0000       102
              Infiltration     1.0000    1.0000    1.0000       107
Web Attack � Sql Injection     1.0000    0.9615    0.9804       104
          Web Attack � XSS     1.0000    0.9740    0.9868       231

                  accuracy                         0.9961      2544
                 macro avg     0.9990    0.9871    0.9929      2544
              weighted avg     0.9961    0.9961    0.9960      2544


📌 Rare 类别测试集表现：
                     Class  Test Samples  Correctly Predicted  Recall
              Infiltration           107                  107  1.0000
          Web Attack � XSS           231                  225  0.9740
Web Attack � Sql Injection           104                  100  0.9615
                Heartbleed           102                  102 

/home/maia-user/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [22:58:04] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



📊 分类报告 (XGBoost ACGAN增强):
                            precision    recall  f1-score   support

                    BENIGN     0.9950    0.9995    0.9973      2000
                Heartbleed     1.0000    1.0000    1.0000       102
              Infiltration     1.0000    0.9907    0.9953       107
Web Attack � Sql Injection     0.9901    0.9615    0.9756       104
          Web Attack � XSS     0.9956    0.9740    0.9847       231

                  accuracy                         0.9953      2544
                 macro avg     0.9961    0.9851    0.9906      2544
              weighted avg     0.9953    0.9953    0.9953      2544


📌 Rare 类别测试集表现：
                     Class  Test Samples  Correctly Predicted  Recall
              Infiltration           107                  106  0.9907
          Web Attack � XSS           231                  225  0.9740
Web Attack � Sql Injection           104                  100  0.9615
                Heartbleed           102                  102